<a href="https://colab.research.google.com/github/koyomin9zx/CS313.K21-Homework/blob/master/step_by_step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#download & extract data
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip
!unzip bank.zip

In [0]:
#import lib
import pandas as pd
from collections import defaultdict

In [0]:
#filter non numeric data
def filter_data(data_frame):
  numeric_data=[key for key, value in data_frame.iteritems() if data_frame[key].dtypes=='int64']
  return numeric_data

In [0]:
def binning_data(data_frame,name_prop,num_bin,binning_type):
  max_prop=data_frame[name_prop].max()
  min_prop=data_frame[name_prop].min()
  distance=round((max_prop-min_prop)/num_bin)
  mark=[min_prop]
  scope=defaultdict(list)

  #Equal-width-binning
  if(binning_type==0):
    while True:
      if(mark[-1]+distance < max_prop and len(mark)<num_bin):
        mark.append(mark[-1]+distance)
      else:
        break
    mark.append(max_prop)

    for j in range(1,len(mark)):
      key='['+str(mark[j-1])+','+str(mark[j])+')'
      scope[key].append(0)
      scope[key].append(name_prop)

    for i in data_frame[name_prop]:
      for j in range(1,len(mark)):
        if(i<mark[j]):
          key=list(scope)[j-1]
          scope[key][0]+=1
          data_frame[name_prop]=data_frame[name_prop].replace(i,key)
          break
        elif (i==max_prop):
          key=list(scope)[-1]
          scope[key][0]+=1
          data_frame[name_prop]=data_frame[name_prop].replace(i,key)
          break
        else:
          continue

  #Equal-depth –binning
  if(binning_type==1):
    values_prop=data_frame[name_prop].values
    values_prop.sort()
    distance=round(len(values_prop)/num_bin)
    
    temp=distance-1
    for i in range(0,num_bin-1):
      mark.append(values_prop[temp])
      temp+=distance
  
    mark.append(max_prop)

    for j in range(1,len(mark)):
      if(j==1):
        key='['+str(mark[j-1])+','+str(mark[j])+']'
      else:
        key='('+str(mark[j-1])+','+str(mark[j])+']'
      scope[key].append(0)
      scope[key].append(name_prop)
    
    for i in data_frame[name_prop]:
      for j in range(1,len(mark)):
        key=list(scope)[j-1]
        if(i<=mark[j]):
          scope[key][0]+=1
          data_frame[name_prop]=data_frame[name_prop].replace(i,key)
          break
        else:
          continue

  log_file="Numeric: "+name_prop+" "
  for key, count in scope.items():
    log_file+=key+":"+str(count[0])+" "


  return data_frame,log_file

In [0]:
def module_iii(input_data,output_data,log_file):
  #read data
  bank_df = pd.read_csv(input_data, encoding='utf-8', sep=',')
  data_frame=bank_df.copy()
  numeric_data=filter_data(bank_df)
  print('==========Danh sách các thuộc tính numeric có trong tập data===========\n')
  for i in range (0,len(numeric_data)):
    print(str(i)+"- "+numeric_data[i])

  try:
    choice = list(map(int,input("\nNhập số thứ tự thuộc tính cần chia giỏ : ").strip().split()))[:len(numeric_data)]
    s= int(input("\nNhập số giỏ cần chia (>2): "))
    t= int(input("\nNhập phương pháp chia giỏ: \n(0) là chia theo chiều rộng, \n(1) là chia theo chiều sâu\n"))
  
  except:
    print("nhập không hợp lệ !!!")   
  log_file1=""
  for i in choice:
    data_frame,log_f=binning_data(data_frame,numeric_data[i],s,t)
    log_file1+=log_f+"\n"

  data_frame.to_csv(output_data,sep=',',index=None)
  with open(log_file, "w", encoding="utf-8") as text_file:
    text_file.write(log_file1)


In [0]:
if __name__ == '__main__': 
  module_iii("newbank.csv","ouput_data.csv","log.txt")
 

==========Danh sách các thuộc tính numeric có trong tập data===========

0- age
1- balance
2- day
3- duration
4- campaign
5- pdays
6- previous

Nhập số thứ tự thuộc tính cần chia giỏ : 0 1 2 3 4 5 6

Nhập số giỏ cần chia (>2): 6

Nhập phương pháp chia giỏ: 
(0) là chia theo chiều rộng, 
(1) là chia theo chiều sâu
0


In [0]:
  test = pd.read_csv('ouput_data.csv', encoding='utf-8', sep=',')
  test.head(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,"[30,41)",unemployed,married,primary,no,"[-3313,9104)",no,no,cellular,"[16,21)",oct,"[4,508)","[1,9)","[-1,144)","[0,4)",unknown,no
1,"[30,41)",services,married,secondary,no,"[-3313,9104)",yes,yes,cellular,"[11,16)",may,"[4,508)","[1,9)","[289,434)","[4,8)",failure,no
2,"[30,41)",management,single,tertiary,no,"[-3313,9104)",yes,no,cellular,"[16,21)",apr,"[4,508)","[1,9)","[289,434)","[0,4)",failure,no
3,"[30,41)",management,married,tertiary,no,"[-3313,9104)",yes,yes,unknown,"[1,6)",jun,"[4,508)","[1,9)","[-1,144)","[0,4)",unknown,no
4,"[52,63)",blue-collar,married,secondary,no,"[-3313,9104)",yes,no,unknown,"[1,6)",may,"[4,508)","[1,9)","[-1,144)","[0,4)",unknown,no


In [0]:
  f = open("log.txt", "r")
  print(f.read())

Numeric: age [19,30):482 [30,41):1950 [41,52):1253 [52,63):732 [63,74):66 [74,87):38 
Numeric: balance [-3313,9104):4410 [9104,21521):94 [21521,33938):15 [33938,46355):1 [46355,58772):0 [58772,71188):1 
Numeric: day [1,6):566 [6,11):770 [11,16):838 [16,21):1039 [21,26):502 [26,31):806 
Numeric: duration [4,508):3945 [508,1012):474 [1012,1516):78 [1516,2020):19 [2020,2524):3 [2524,3025):2 
Numeric: campaign [1,9):4334 [9,17):144 [17,25):29 [25,33):12 [33,41):0 [41,50):2 
Numeric: pdays [-1,144):3922 [144,289):336 [289,434):242 [434,579):14 [579,724):3 [724,871):4 
Numeric: previous [0,4):4297 [4,8):172 [8,12):35 [12,16):9 [16,20):3 [20,25):5 

